In [43]:
using CSV, DataFrames, Statistics, Random, Plots, JuMP, Gurobi
using ScikitLearn
@sk_import linear_model: LinearRegression
@sk_import linear_model: Lasso
@sk_import model_selection: train_test_split
@sk_import preprocessing: StandardScaler

┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore ~/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:259


PyObject <class 'sklearn.preprocessing._data.StandardScaler'>

In [44]:
function opt_split(X, y, n, p, k, lambda, weight = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)
    p = p + 1

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, theta)
    @variable(model, u[1:n] >= 0)
    @variable(model, beta[1:p])
    @variable(model, w[1:p])

    @objective(model, Min, k * theta + sum(u) + lambda * sum(w))

    for i in 1:p
        @constraint(model, w[i] >= beta[i])
        @constraint(model, w[i] >= -beta[i])
    end 

    for i in 1:n
        if weight == nothing
            @constraint(model, theta + u[i] >= y[i] - sum(X[i, :].*beta))
            @constraint(model, theta + u[i] >= -(y[i] - sum(X[i, :].*beta)))
        else 
            @constraint(model, theta + u[i] >=  weight[i] * (y[i] - sum(X[i, :].*beta)))
            @constraint(model, theta + u[i] >= - weight[i] * (y[i] - sum(X[i, :].*beta)))
        end 
    end
    
    optimize!(model)

    return value(theta), value.(u), value.(beta), value.(w)
end

opt_split (generic function with 2 methods)

In [45]:
function split_train_val(X, y, beta_star, train_fraction = 0.8, weights = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)

    residuals = y - X * beta_star

    if weights == nothing
        sorted_indices = sortperm(abs.(residuals), rev=true)
    else 
        residuals_weights = [residuals[i] * weights[i] for i in 1:length(weights)]
        sorted_indices = sortperm(abs.(residuals_weights), rev=true)
    end

    num_train_points = round(Int, train_fraction * length(sorted_indices))

    train_indices = sorted_indices[1:num_train_points]

    val_indices = setdiff(1:length(y), train_indices)

    X_train = X[train_indices, 2:end]
    y_train = y[train_indices, :]

    X_val = X[val_indices, 2:end]
    y_val = y[val_indices, :]

    return X_train, y_train, X_val, y_val
end

split_train_val (generic function with 3 methods)

In [46]:
function fit_evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test, lambdas)
    current_lambda = Inf
    current_val_mse = Inf
    current_test_mse = Inf
    
    for lambda in lambdas

        model = Lasso(alpha=lambda, max_iter = 20000)
        fit!(model, X_train, y_train)
        
        y_pred_val = predict(model, X_val)
        val_mse_i = mse(y_val, y_pred_val)
        
        if current_val_mse >= val_mse_i
            current_lambda = lambda
            current_val_mse = val_mse_i
            current_test_mse = mse(y_test, predict(model, X_test))
        end
    end

    println("best lambda: ", current_lambda)
    println("val score: ", current_val_mse)
    println("test score: ", current_test_mse)

    return current_val_mse, current_test_mse
end

fit_evaluate_model (generic function with 1 method)

In [47]:
function mse(y_true, y_pred)
    mse = mean((y_true .- y_pred).^2)
    return mse
end

mse (generic function with 1 method)

## Comp Hard dataset

In [48]:
using CSV
using DataFrames

# Load the CSV file into a DataFrame
df = CSV.read("data/comp_hard/machine.data", header = false, DataFrame)
df_names = ["Vendor_Name", "Model_Name", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]
rename!(df, Symbol.(df_names))

feature_list = ["MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP"]
X = df[:, feature_list]
y = df[!, "ERP"];

In [49]:
function normalize_data(X_train, X_test)
    # Calculate mean and standard deviation from training data
    mean_vals = mean(X_train, dims=1)
    std_vals = std(X_train, dims=1)

    # Normalize training data
    X_train_norm = (X_train .-mean_vals) ./ std_vals;
    X_test_norm = (X_test .-mean_vals) ./ std_vals;

    return X_train_norm, X_test_norm
end

normalize_data (generic function with 1 method)

In [50]:
(train_X, train_y), (test_X, test_y) =
    IAI.split_data(:regression, X, y, train_proportion=0.9);

In [51]:
train_X_norm, test_X_norm = normalize_data(Matrix(train_X), Matrix(test_X));

In [52]:
#get optimised split 
n = nrow(train_X)
p = ncol(train_X)
train_fraction = 0.7
k = n * train_fraction
lambda = 0.001;

In [53]:
#get optimised split
_, _, betas_star, _ = opt_split(train_X_norm, Array(train_y), n, p, k, lambda)
X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(train_X_norm, Array(train_y), betas_star);

#get non optimised split 
(X_train_rand, y_train_rand), (X_val_rand, y_val_rand) =
    IAI.split_data(:regression, train_X_norm, train_y, train_proportion=train_fraction);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [54]:
lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1]
fit_evaluate_model(X_train_opt, X_val_opt, Matrix(test_X_norm), y_train_opt, y_val_opt, Array(test_y), lambdas)

best lambda: 0.1
val score: 129.74951750953483
test score: 1255.246060821048


(129.74951750953483, 1255.246060821048)

In [55]:
fit_evaluate_model(Matrix(X_train_rand), Matrix(X_val_rand), Matrix(test_X_norm), Array(y_train_rand), y_val_rand, Array(test_y), lambdas)

best lambda: 0.1
val score: 1268.7514238010428
test score: 1522.9794563294136


(1268.7514238010428, 1522.9794563294136)